In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from itertools import chain
import nltk
import sklearn
import scipy. stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import pandas as pd
import numpy as np
from itertools import groupby
import pickle

load data

In [2]:
train=pd.read_table('训练语料/train/train_data.txt',names=['id','word','entity','tag'])
test=pd.read_table('训练语料/test/test_data.txt',names=['id','word','entity','tag'])
validf=pd.read_table('训练语料/test/test_data_label.txt', names=['id','word','entity','tag'])

In [3]:
train=train.dropna(subset=['word'])

In [4]:
len(train), len(test), len(validf)

(32971, 14712, 14712)

In [5]:
def remove_56_motor(df):
    l=df['tag']
    ldx56motor=[]
    for i in range(len(df)):
        if '56_Motor' in l[i] or 'TemporalConstraint' in l[i]:
            ldx56motor.append(i)
    newdf=df.drop(ldx56motor, axis=0)
    return newdf

In [6]:
train=remove_56_motor(train)
validf=remove_56_motor(validf)
print(len(train),len(validf))

32664 14632


In [7]:
labels=list(set(train['tag']))
labels.remove('O')
labels

['I-1a_LOC',
 'I-7_LimbAtaxia',
 'I-2_BestGaze',
 'I-6_Motor',
 'B-6a_LeftLeg',
 'I-10_Dysarthria',
 'B-9_BestLanguage',
 'B-6_Motor',
 'I-NIHSS',
 'B-7_LimbAtaxia',
 'I-6a_LeftLeg',
 'I-5a_LeftArm',
 'I-Measurement',
 'B-5a_LeftArm',
 'B-4_FacialPalsy',
 'I-1b_LOCQuestions',
 'I-8_Sensory',
 'I-6b_RightLeg',
 'I-9_BestLanguage',
 'B-Measurement',
 'I-4_FacialPalsy',
 'B-3_Visual',
 'B-11_ExtinctionInattention',
 'B-1b_LOCQuestions',
 'I-5_Motor',
 'I-5b_RightArm',
 'B-1a_LOC',
 'B-8_Sensory',
 'B-10_Dysarthria',
 'B-6b_RightLeg',
 'B-5b_RightArm',
 'B-5_Motor',
 'B-NIHSS',
 'I-3_Visual',
 'B-1c_LOCCommands',
 'I-1c_LOCCommands',
 'B-2_BestGaze',
 'I-11_ExtinctionInattention']

In [8]:
label_set=labels

In [9]:
def dataset_id_list(pkl_file_type,num):
    pkl_file_name = 'nihss_'+ pkl_file_type +'_fold_'+str(num)
    with open('训练语料/train/' + pkl_file_name + '.pkl', 'rb') as f:
        nihss_train_fold_0 = pickle.load(f)
    id_list = []
    for i in nihss_train_fold_0:
        x = i['HADM_ID']
        id_list.append(x)
    return id_list

In [10]:
# training set
traindf0 = train[train.id.isin(dataset_id_list('train',0))]
traindf1 = train[train.id.isin(dataset_id_list('train',1))]
traindf2 = train[train.id.isin(dataset_id_list('train',2))]
traindf3 = train[train.id.isin(dataset_id_list('train',3))]
traindf4 = train[train.id.isin(dataset_id_list('train',4))]

# validating set
testdf0 = train[train.id.isin(dataset_id_list('valid',0))]
testdf1 = train[train.id.isin(dataset_id_list('valid',1))]
testdf2 = train[train.id.isin(dataset_id_list('valid',2))]
testdf3 = train[train.id.isin(dataset_id_list('valid',3))]
testdf4 = train[train.id.isin(dataset_id_list('valid',4))]

In [11]:
traindf0.head()

,id,word,entity,tag
0,150239,an,NaN,O
1,150239,"""embolic",NaN,O
2,150239,"stroke""",NaN,O
3,150239,<CRLF>,NaN,O
4,150239,Physical,NaN,O


In [12]:
traindf0.head()

,id,word,entity,tag
0,150239,an,NaN,O
1,150239,"""embolic",NaN,O
2,150239,"stroke""",NaN,O
3,150239,<CRLF>,NaN,O
4,150239,Physical,NaN,O


df to list

In [13]:
def df2list2(df):
    train_sents=[]
    for i in set(df['id']):
        ltrain=[]
        sdf=df[df['id']==i]
        for j in range(len(sdf)):
            s=tuple(sdf.iloc[j,1:4])
            #print(s)
            ltrain.append(s)
        train_sents.append(ltrain)
    return train_sents

In [14]:
def word2features(sent,i):
    word=sent[i][0]
    postag=sent[i][1]
    
    features={
        'bias':1.0,
        'word.lower()':word.lower(),
        'word[-3:]':word[-3:],
        'word[-2:]':word[-2:],
        'word.isupper()':word.isupper(),
        'word.istitle()':word.istitle(),
        'word.isdigit()':word.isdigit(),
        #'postag[:2]':postag[:2],
    }
    if i>0:
        word1=sent[i-1][0]
        postag1=sent[i-1][1]
        features.update({
            '-1:word.lower()':word1.lower(),
            '-1:word.istitle()':word1.istitle(),
            '-1:word.isupper()':word1.isupper(),
            #'-1:postag':postag1,
            #'-1:postag[:2]':postag1[:2],
        })
    else:
        features['BOS']=True
        
    if i<len(sent)-1:
        word1=sent[i+1][0]
        #print(i,word1)
        postag1=sent[i+1][0]
        features.update({
            '+1:word.lower()':word1.lower(),
            '+1:word.istitle()':word1.istitle(),
            '+1:word.isupper()':word1.isupper(),
            #'+1:postag':postag1,
            #'+1:postag[:2]':postag1[:2],
        })
    else:
        features['EOS']=True
    
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [15]:
vali_set=df2list2(validf)
vali_set

[[('the', nan, 'O'),
  ('time', nan, 'O'),
  ('.', nan, 'O'),
  ('<CRLF>', nan, 'O'),
  ('Patient', nan, 'O'),
  ('was', nan, 'O'),
  ('evaluated', nan, 'O'),
  ('and', nan, 'O'),
  ('had', nan, 'O'),
  ('an', nan, 'O'),
  ('NIHSS', 'T1', 'B-NIHSS'),
  ('4', 'T2', 'B-Measurement'),
  ('.', nan, 'O'),
  ('Per', nan, 'O'),
  ('radiology', nan, 'O'),
  (',', nan, 'O'),
  ('the', nan, 'O'),
  ('<CRLF>', nan, 'O'),
  ('well', nan, 'O'),
  ('-', nan, 'O'),
  ('circumscribed', nan, 'O'),
  ('hyderdense', nan, 'O')],
 [('3105', nan, 'O'),
  ('*', nan, 'O'),
  ('*', nan, 'O'),
  (']', nan, 'O'),
  (',', nan, 'O'),
  ('where', nan, 'O'),
  ('he', nan, 'O'),
  ('<CRLF>', nan, 'O'),
  ('received', nan, 'O'),
  ('an', nan, 'O'),
  ('NIHSS', 'T1', 'B-NIHSS'),
  ('of', nan, 'O'),
  ('9', 'T2', 'B-Measurement'),
  ('(', nan, 'O'),
  ('breakdown', nan, 'O'),
  ('not', nan, 'O'),
  ('available', nan, 'O'),
  ('on', nan, 'O'),
  ('the', nan, 'O'),
  ('provided', nan, 'O'),
  ('<CRLF>', nan, 'O'),
  ('not

extract features from data

training

In [17]:
def training(X_train,y_train):
    %%time
    crf=sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=True
    )
    crf.fit(X_train, y_train)
    return crf

evaluation

In [18]:
def sent2tag(sent_list):
    sent2tag_list=[]
    for i in sent_list:
        sent2tag_list=sent2tag_list+i
    return sent2tag_list

def tag2entity(sent_list):
    tag2entity_list=[]
    for j in sent2tag(sent_list):
        if j=='O':
            tag2entity_list.append([j])
        elif j[0]=='B':
            tag2entity_list.append([j])
        elif j[0]=='I':
            tag2entity_list[len(tag2entity_list)-1].append(j)
    return tag2entity_list

#def tag2entity_pred(sent_list,y_pred):
#    tag2entity_pred_list=[]
#    x=0
#    for i in tag2entity(sent_list):
#        tag2entity_pred_list.append(sent2tag(y_pred)[x:x+len(i)])
#        x+=len(i)
#    return tag2entity_pred_list
        
def entity2label(tag2entity_list):
    entity2label_list=[]
    for k in tag2entity_list:
        #print(k)
        l=[]
        for m in k:
            lm=m.split('-')
            l.append(lm[len(lm)-1])
        if len(set(l))==1:
            entity2label_list.append(l[0])
        elif len(set(l))>1:
            entity2label_list.append(','.join(set(l)))  
            #print(','.join(set(l)))
    return entity2label_list

In [19]:
def tag2entity_pred(sent_list_test, sent_list_pred):
    tag2entity_pred_list=[]
    tag2entity_list=tag2entity(sent_list_test)
    sent2tag_pred_list=sent2tag(sent_list_pred)
    x=0
    for i in tag2entity_list:
        tag2entity_pred_list.append(sent2tag_pred_list[x:x+len(i)])
        x+=len(i)
    return tag2entity_pred_list

calculate precision recall

precision: corrected predicted nihss entitiy / all entity predicted as nihss

In [20]:
def overall_perf(test_label,pred_label):
    tp=0
    for i in range(len(test_label)):
        if test_label[i]==pred_label[i] and test_label[i]!='O':
            tp+=1
    while 'O' in test_label:
        test_label.remove('O')
    rp=len(test_label)
    
    while 'O' in pred_label:
        pred_label.remove('O')
    pp=len(pred_label)
    precision=tp/pp
    recall=tp/rp
    f1=2*(precision*recall)/(precision+recall)
    overall_df=pd.DataFrame(data={'precision':precision, 'recall':recall, 'f1':f1}, index=['overall'])
    return overall_df

In [21]:
def train_test_vali(traindf,testdf,validf):
    train_set=df2list2(traindf)
    test_set=df2list2(testdf)
    vali_set=df2list2(validf)
    
    X_train=[sent2features(s) for s in train_set]
    y_train=[sent2labels(s) for s in train_set]
    X_test=[sent2features(s) for s in test_set]
    y_test=[sent2labels(s) for s in test_set]
    X_vali=[sent2features(s) for s in vali_set]
    y_vali=[sent2labels(s) for s in vali_set]
    
    #crf=sklearn_crfsuite.CRF(
        #algorithm='lbfgs',
        #c1=0.1,
        #c2=0.1,
        #max_iterations=100,
        #all_possible_transitions=True
    #)
    #crf.fit(X_train, y_train)
    
    #labels=list(crf.classes_)
    #labels.remove('O')
    #label_set=[x.split('-')[1] for x in labels]
    label_set=list(set(traindf['tag']))
    label_set.remove('O')
    
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs', 
        max_iterations=100, 
        all_possible_transitions=True
    )
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
    }
    
    # use the same metric for evaluation
    f1_scorer = make_scorer(metrics.flat_f1_score, 
                            average='weighted', labels=label_set)

    # search
    rs = RandomizedSearchCV(crf, params_space, 
                            #cv=3,  #cv default value is 5
                            verbose=1, 
                            n_jobs=-1, 
                            n_iter=50, 
                            scoring=f1_scorer)
    rs.fit(X_train, y_train)
    crf=rs.best_estimator_
    y_pred_vali=crf.predict(X_vali)

    #y_pred=crf.predict(X_test)
    #f1_score=metrics.flat_f1_score(y_test, y_pred, average='weighted',labels=labels)
    #print('f1:',f1_score)
    
    y_vali_label=entity2label(tag2entity(y_vali))
    y_pred_vali_label=entity2label(tag2entity_pred(y_vali, y_pred_vali))
    
    performance=perfm(y_vali_label,y_pred_vali_label)
    overalldf=overall_perf(y_vali_label,y_pred_vali_label)
    performance=performance.append(overalldf)
    
    return performance

In [22]:
def perfm(y_real,y_pred):
    TP_dict=dict()
    PP_dict=dict()
    RP_dict=dict()
    label_set = list(set(y_real))
    label_set.remove('O')
    
    for i in [TP_dict,PP_dict, RP_dict]:
        for j in label_set:
            i[j]=0
    
    for i in range(len(y_real)):
        if y_real[i]==y_pred[i] and y_real[i]!='O':
            TP_dict[y_real[i]]+=1
    for i in y_pred:
        if i in label_set:
            PP_dict[i]+=1
    #print(PP_dict)
    for i in y_real:
        if i in label_set:
            RP_dict[i]+=1
            
    precision_dict=dict()
    recall_dict=dict()
    f1_dict=dict()
    for i in label_set:
        if PP_dict[i]==0:
            precision_dict[i]=0
        else:   
            precision_dict[i]=TP_dict[i]/PP_dict[i]
        if RP_dict[i]==0:
            recall_dict[i]=0
        else:
            recall_dict[i]=TP_dict[i]/RP_dict[i]
        if (precision_dict[i]+recall_dict[i])==0:
            f1_dict[i]=0
        else:
            f1_dict[i]=2*(precision_dict[i]*recall_dict[i])/(precision_dict[i]+recall_dict[i])
    
    precision_df=pd.DataFrame(precision_dict, index=['precision'])
    recall_df=pd.DataFrame(recall_dict, index=['recall'])
    f1_df=pd.DataFrame(f1_dict, index=['f1'])
    perf_df=pd.concat([precision_df, recall_df,f1_df])
    perf_dft=pd.DataFrame(perf_df.values.T, index=perf_df.columns, columns=perf_df.index)
    return perf_dft

In [23]:
train_test_vali(traindf1,testdf1,validf)

D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  9.4min finished


,precision,recall,f1
3_Visual,0.913043,0.913043,0.913043
1b_LOCQuestions,1.000000,0.933333,0.965517
4_FacialPalsy,0.875000,0.859649,0.867257
6b_RightLeg,0.939394,0.911765,0.925373
10_Dysarthria,0.852459,0.945455,0.896552
5b_RightArm,0.846154,0.825000,0.835443
8_Sensory,0.872340,0.820000,0.845361
6a_LeftLeg,1.000000,0.923077,0.960000
9_BestLanguage,0.934783,0.860000,0.895833
5_Motor,1.000000,0.500000,0.666667


In [38]:
y=0
for i,j,k in [(traindf0,testdf0,'fold0'),(traindf1,testdf1,'fold1'),(traindf2,testdf2,'fold2'),(traindf3,testdf3,'fold3'),(traindf4,testdf4,'fold4')]:
    print(y)
    if y==0:
        df=train_test_vali(i,j,validf)
        y+=1
    else:
        dfy=train_test_vali(i,j,validf)
        dfy.columns=[i+k for i in dfy.columns]
        df=pd.concat([df,dfy], axis=1, join_axes=[df.index])
        y+=1
df

0


D:\Anaconda\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  1.9min finished


1
Fitting 5 folds for each of 50 candidates, totalling 250 fits


...

3
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.2min finished


4
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.7min finished


,precision,recall,f1,precisionfold1,recallfold1,f1fold1,precisionfold2,recallfold2,f1fold2,precisionfold3,recallfold3,f1fold3,precisionfold4,recallfold4,f1fold4
5b_RightArm,0.966667,0.725000,0.828571,0.970588,0.825000,0.891892,0.966667,0.725000,0.828571,0.702128,0.825000,0.758621,0.783784,0.725000,0.753247
10_Dysarthria,0.941176,0.872727,0.905660,0.976744,0.763636,0.857143,0.937500,0.818182,0.873786,0.958333,0.836364,0.893204,0.909091,0.909091,0.909091
7_LimbAtaxia,1.000000,0.928571,0.962963,1.000000,0.928571,0.962963,0.709091,0.928571,0.804124,1.000000,0.952381,0.975610,1.000000,0.952381,0.975610
6_Motor,0.800000,0.444444,0.571429,1.000000,0.444444,0.615385,1.000000,0.444444,0.615385,0.000000,0.000000,0.000000,1.000000,0.444444,0.615385
5_Motor,1.000000,0.500000,0.666667,1.000000,0.500000,0.666667,1.000000,0.500000,0.666667,0.000000,0.000000,0.000000,1.000000,0.500000,0.666667
9_BestLanguage,0.953488,0.820000,0.881720,0.860000,0.860000,0.860000,0.875000,0.840000,0.857143,0.951220,0.780000,0.857143,0.950000,0.760000,0.844444
3_Visual,1.000000,0.847826,0.917647,1.000000,0.847826,0.917647,0.891304,0.891304,0.891304,0.886364,0.847826,0.866667,1.000000,0.804348,0.891566
6a_LeftLeg,1.000000,0.769231,0.869565,0.894737,0.871795,0.883117,1.000000,0.794872,0.885714,0.891892,0.846154,0.868421,1.000000,0.769231,0.869565
6b_RightLeg,0.965517,0.823529,0.888889,0.969697,0.941176,0.955224,0.756757,0.823529,0.788732,0.968750,0.911765,0.939394,0.933333,0.823529,0.875000
4_FacialPalsy,0.750000,0.842105,0.793388,0.928571,0.684211,0.787879,0.977273,0.754386,0.851485,0.891304,0.719298,0.796117,0.954545,0.736842,0.831683


In [1]:
df['average_precision']=df.apply(lambda x: (x.precisionfold0+x.precisionfold1+x.precisionfold2+x.precisionfold3+x.precisionfold4)/5)
df['average_recall']=df.apply(lambda x: (x.recallfold0+x.recallfold1+x.recallfold2+x.recallfold3+x.recallfold4)/5)
df['average_f1']=df.apply(lambda x: (x.f1fold0+x.f1fold1+x.f1fold2+x.f1fold3+x.f1fold4)/5)

NameError: name 'df' is not defined

In [27]:
df.to_csv('performance_vali_without56motor_tempreal_constraint test cv3 9 18.csv')

In [ ]:
%%time
crf=sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space={
    'c1':scipy.stats.expon(scale=0.5),
    'c2':scipy.stats.expon(scale=0.05),
}

f1_scorer=make_scorer(metrics.flat_f1_score,average='weighted',labels=labels)

rs=RandomizedSearchCV(crf,params_space, cv=3, verbose=1, n_jobs=-1, n_iter=50, scoring=f1_scorer)

rs.fit(X_train, y_train)

In [ ]:
print('best params:', rs.best_params_)
print('best cv score', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_/1000000))

check parameter space

In [ ]:
[s for s in rs.cv_results_]

In [ ]:
#_x = [s.parameters['param_c1'] for s in rs.cv_results_]
#_y = [s.parameters['param_c2'] for s in rs.cv_results_]
#_c = [s.mean_validation_score for s in rs.cv_results_]
_x=rs.cv_results_['param_c1']
_y=rs.cv_results_['param_c2']
_c=rs.cv_results_['mean_test_score']

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

check best estimator on test data

In [ ]:
crf=rs.best_estimator_
y_pred=crf.predict(X_test)
print(metrics.flat_classification_report(y_test,y_pred,labels=sorted_labels,digits=3))

check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print('%-6s -> %-7s %0.6f' % (label_from, label_to,weight))
        
print('top likely transitions:')
print_transitions(Counter(crf.transition_features_).most_common(20))

print('\nTop unlikely transitions:')
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

check state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print('%0.6f %-8s %s' % (weight, label, attr))
        
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])  

In [7]:
code=['0']*10
for i in [('T11', 'Measurement', 0, 3), ('T12', '4_FacialPalsy', 6, 6)]:
    t=i[0]
    x=i[2]
    y=i[3]
    for j in range(x,y+1):
        code[j]=t
code

T11
T12


['T11', 'T11', 'T11', 'T11', '0', '0', 'T12', '0', '0', '0']